In [2]:
import pandas as pd 
import numpy as np 
from transformers import AutoTokenizer
from transformers import pipeline
from datasets import DatasetDict
import os
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint

In [3]:
df = pd.read_csv('data/news_summary.csv',encoding='utf-8',encoding_errors = 'replace')

In [19]:
from torch import cuda 
device = 'cuda' if cuda .is_available() else 'cpu'
device

'cpu'

In [4]:
df.head()

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [5]:
df = df.drop(columns=['author','date','read_more'])
df

,headlines,text,ctext
0,Daman & Diu revokes mandatory Rakshabandhan in...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Malaika slams user who trolled her for 'divorc...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,'Virgin' now corrected to 'Unmarried' in IGIMS...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Aaj aapne pakad liya: LeT man Dujana before be...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotel staff to get training to spot signs of s...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...
...,...,...,...
4509,Rasna seeking ?250 cr revenue from snack categ...,Fruit juice concentrate maker Rasna is eyeing ...,"Mumbai, Feb 23 (PTI) Fruit juice concentrate m..."
4510,Sachin attends Rajya Sabha after questions on ...,Former Indian cricketer Sachin Tendulkar atten...,Former cricketer Sachin Tendulkar was spotted ...
4511,Shouldn't rob their childhood: Aamir on kids r...,"Aamir Khan, while talking about reality shows ...","Aamir Khan, whose last film Dangal told the st..."
4512,"Asha Bhosle gets ?53,000 power bill for unused...",The Maharashtra government has initiated an in...,Maharahstra Power Minister Chandrashekhar Bawa...


In [6]:
##Checking null values in the dataset
df.isnull().sum()

headlines      0
text           0
ctext        118
dtype: int64

In [7]:
##Dropping the columns with null values
df = df.dropna()
df.head()

,headlines,text,ctext
0,Daman & Diu revokes mandatory Rakshabandhan in...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Malaika slams user who trolled her for 'divorc...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,'Virgin' now corrected to 'Unmarried' in IGIMS...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Aaj aapne pakad liya: LeT man Dujana before be...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotel staff to get training to spot signs of s...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [8]:
#adding the headlines and ctext column and making a new news column
df['news'] = (df['headlines'] + '.' + df['ctext']).apply(str.lower)
df = df.drop(columns=['ctext','headlines'])
df= df.rename(columns={'text':'summary'})
df['summary']=df['summary'].apply(str.lower)
df.head()

/var/folders/5k/wmytnlns4sd2bm57pyry3xxr0000gn/T/ipykernel_32215/2204458343.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['news'] = (df['headlines'] + '.' + df['ctext']).apply(str.lower)


,summary,news
0,the administration of union territory daman an...,daman & diu revokes mandatory rakshabandhan in...
1,malaika arora slammed an instagram user who tr...,malaika slams user who trolled her for 'divorc...
2,the indira gandhi institute of medical science...,'virgin' now corrected to 'unmarried' in igims...
3,lashkar-e-taiba's kashmir commander abu dujana...,aaj aapne pakad liya: let man dujana before be...
4,hotels in maharashtra will train their staff t...,hotel staff to get training to spot signs of s...


splitting the data into train and test

In [9]:
from sklearn.model_selection import train_test_split
import datasets
# Converting the pandas dataset to huggingface dataset
# first split the train and test set
train_df, test_df = train_test_split(df, test_size=0.01, shuffle=True)
print("train and val shape:", train_df.shape, "test shape:",test_df.shape)
# save for every model inference
global_train_df = train_df
global_test_df = test_df
train_df = datasets.Dataset.from_pandas(train_df)
train_df = train_df.remove_columns(["__index_level_0__"])
# split train into train and val
train_df = train_df.train_test_split(test_size=0.2, shuffle=True) # split train and test
train_df["train"][0]





train and val shape: (4352, 2) test shape: (44, 2)


{'summary': 'after bjp formed the government in up, posters of bjp mla brijesh singh surfaced in saharanpur naming deoband as deovrind. notably, deoband is known for its historical significance and darul uloom deoband ? a centre of islamic learning from mughal era. "we want all those places to be renamed which were once under the mughal rule," said a vhp leader.',
 'news': 'bjp mla\'s posters in up mention deoband as deovrind.bjp mla brijesh singh\'s posters thanking voters for the recent astounding victory have created a stir in the saharanpur district of uttar pradesh after they named deoband as deovrind. while deoband is known for its historical significance and darul uloom deoband has been a centre of islamic learning since the mughal era, brijesh singh had demanded that the city be renamed deovrind after bjp came into power. "i have no such intention that can connect it to any islamic institution. all i want is to rename deoband to deovrind. it is all about development", said sing

In [10]:
checkpoint = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [11]:
prefix = "summarize: "

def preprocess(news):
    inputs = [prefix + doc for doc in news['news']]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)  # for news data
    labels = tokenizer(news['summary'], max_length=128, truncation=True)  # for summary
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [27]:
# Fitting into dataset dict
train_val_test_dataset = DatasetDict({
    'train': train_df["train"],
    'val': train_df['test']})

print(type(train_val_test_dataset))
train_val_test_dataset

<class 'datasets.dataset_dict.DatasetDict'>


DatasetDict({
    train: Dataset({
        features: ['summary', 'news'],
        num_rows: 3481
    })
    val: Dataset({
        features: ['summary', 'news'],
        num_rows: 871
    })
})

In [39]:
tokenized_data = train_val_test_dataset.map(preprocess,batched= True)






Map: 100%|██████████| 3481/3481 [00:01<00:00, 2243.21 examples/s]


Map: 100%|██████████| 871/871 [00:00<00:00, 2218.17 examples/s]


In [40]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,model=checkpoint)

In [41]:
import evaluate
rouge = evaluate.load("rouge")

In [42]:
##evaluating the rouge matrix 
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    # result["gen_len"] = np.mean(prediction_lens)

    # return {k: round(v, 4) for k, v in result.items()}
    

In [46]:
from transformers import AutoModelForSeq2SeqLM,Seq2SeqTrainer,Seq2SeqTrainingArguments
model =AutoModelForSeq2SeqLM.from_pretrained(checkpoint).to(device)

In [54]:
training_args = Seq2SeqTrainingArguments(
    output_dir="news summarization",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=False,
)


In [55]:

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

  0%|          | 1/3484 [01:24<81:47:54, 84.55s/it]

KeyboardInterrupt: 